In [ ]:
from typing import Optional, Annotated

from pydantic import BaseModel, Field

class LaxInt(BaseModel):
    x: Annotated[int, Field()]

class StrictInt(BaseModel):
    x: Annotated[int, Field(strict=True)]

class LaxFloat(BaseModel):
    x: Annotated[float, Field()]

class StrictFloat(BaseModel):
    x: Annotated[float, Field(strict=True)]

class StrictTupleOfInt(BaseModel):
    x: tuple[Annotated[int, Field(strict=True)], ...]
    y: Optional[tuple[Annotated[int, Field(strict=True)], ...]] = None

class EmptyTupleTests(BaseModel):
    strings: Optional[tuple[str, ...]] = None
    bools: Optional[tuple[bool, ...]] = None


In [64]:
foo = EmptyTupleTests()
foo.strings = tuple()
foo.bools = tuple()

In [ ]:
StrictTupleOfInt.model_json_schema()

In [ ]:
foo4 = StrictTupleOfInt(x=(1,), y=(2,3))
foo5 = StrictTupleOfInt(x=(1,), y=None, z=(1))

In [ ]:
foo0 = LaxInt(x=1.0)

In [ ]:
foo1 = StrictInt(x=1.0)

In [ ]:
foo2 = LaxFloat(x=1)

In [ ]:
foo3 = StrictFloat(x=1)